In [1]:
import pandas as pd
import os
df=pd.read_excel("Data_For Model 20230107.xlsx")

In [ ]:
import json
import numpy as np

from bert4keras.backend import keras, search_layer, K
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from tensorflow.keras.layers import Lambda, Dense,Bidirectional,LSTM
from tensorflow.keras.losses import kullback_leibler_divergence as kld
from tqdm import tqdm
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
labels = [0,1]

In [2]:

df["Label"].value_counts()

Label
IS    8131
ES    3093
iS       1
b        1
Name: count, dtype: int64

In [3]:
tmpd={"IS":0,"ES":1}

In [4]:
df["label"]=df["Label"].map(tmpd)

In [5]:
df=df.dropna(subset=["label"])

In [6]:
df["label"].value_counts()

label
0.0    8131
1.0    3093
Name: count, dtype: int64

In [7]:
df["text"]=df["Text"]
df=df.dropna(subset=["text","label"])

In [8]:
df

,Unnamed: 0,Text,Label,label,text
0,NaN,别担心，这种情况是可以解决的,ES,1.0,别担心，这种情况是可以解决的
1,NaN,遇到这种事情是很麻烦的，不是三言两语可以解决的，要具体问题具体对待，希望你们和睦相处，开心快乐！,ES,1.0,遇到这种事情是很麻烦的，不是三言两语可以解决的，要具体问题具体对待，希望你们和睦相处，开心快乐！
2,NaN,我的已经到了，希望大家的也马上到达。,ES,1.0,我的已经到了，希望大家的也马上到达。
3,NaN,呵呵，人力搜索中——查找！,IS,0.0,呵呵，人力搜索中——查找！
4,NaN,寻找共同爱好，比如游戏等,IS,0.0,寻找共同爱好，比如游戏等
...,...,...,...,...,...
11221,NaN,这个也很好玩耶！,ES,1.0,这个也很好玩耶！
11222,NaN,太强了,ES,1.0,太强了
11223,NaN,我非常赞同这位同学的看法,ES,1.0,我非常赞同这位同学的看法
11224,NaN,我觉得您的想法和思维导图做的不错！,ES,1.0,我觉得您的想法和思维导图做的不错！


#### 去除停用词

In [9]:
import re
import jieba_fast as jieba
ting=[i.strip() for i in open("stop_words_ch-停用词表.txt",encoding='gbk').readlines()]
def fenci(text):
    return [i for i in jieba.cut(str(text)) if i not in ting] 

df["text"]=df["text"].apply(lambda x:fenci(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jhg\AppData\Local\Temp\jieba.cache
Loading model cost 0.882 seconds.
Prefix dict has been built succesfully.


In [10]:
df["text"]=df["text"].apply(lambda x:" ".join(x))

### 加载数据

In [ ]:
import pandas as pd
num_classes = len(labels)
maxlen = 64

# BERT base
config_path = 'chinese_roberta_www_ext/bert_config.json'
checkpoint_path = 'chinese_roberta_www_ext/bert_model.ckpt'
dict_path = 'chinese_roberta_www_ext/vocab.txt'

from tqdm import tqdm

In [ ]:
def load_data(df):
    """加载数据
    单条格式：(文本, 标签id)
    """

    D = []
    for i,row in tqdm(df.iterrows()):
        D.append((row["text"], labels.index(row["label"])))
    return D

df=df.dropna(subset=["text"])
# 加载数据集
df=load_data(df)

### 数据加载器

In [ ]:
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
#             print(text,text1,label)
            token_ids, segment_ids = tokenizer.encode(text, maxlen=128)
            for i in range(1):
                batch_token_ids.append(token_ids)
                batch_segment_ids.append(segment_ids)
                batch_labels.append([int(label)])
            if len(batch_token_ids) == self.batch_size  or is_end:
                batch_token_ids = sequence_padding(batch_token_ids,length=128)
                batch_segment_ids = sequence_padding(batch_segment_ids,length=128)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

### 模型搭建

In [ ]:
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    return_keras_model=False,
)
output1 = Lambda(lambda x: x[:, 0])(bert.model.output)
# tmp=bert.model.output

In [ ]:
# lstmout = Bidirectional(LSTM(128, return_sequences=False))(tmp)
output = Dense(
    units=len(labels),
    activation='softmax',
    kernel_initializer=bert.initializer
)(output1)

In [ ]:
model = keras.models.Model(bert.model.input, output)
model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=Adam(2e-5),
    metrics=['sparse_categorical_accuracy']
)
def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total


class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(test_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('bert1.weights'.format(1))
        print(
            u'val_acc: %.5f, best_val_acc: %.5f\n' %
            (val_acc, self.best_val_acc)
        )




### 交叉验证

In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import GaussianNB
kf = KFold(n_splits=5, shuffle=True, random_state=42)

### 每一折的交叉验证报告 手动平均即为交叉验证的平均结果

In [14]:
X=np.array(df)
true_labels_all = []
predicted_labels_all = []
for train_index, test_index in kf.split(X):
    
    train_data,test_data=X[train_index],X[test_index]
    
    bert = build_transformer_model(
        config_path=config_path,
        checkpoint_path=checkpoint_path,
        return_keras_model=False,
    )
    output1 = Lambda(lambda x: x[:, 0])(bert.model.output)
    # tmp=bert.model.output
    
    output = Dense(
        units=len(labels),
        activation='softmax',
        kernel_initializer=bert.initializer
    )(output1)
    
    model = keras.models.Model(bert.model.input, output)
    model.compile(
        loss='sparse_categorical_crossentropy',
        # optimizer=Adam(1e-5),  # 用足够小的学习率
        optimizer=Adam(2e-5),
        metrics=['sparse_categorical_accuracy']
    )
    
    train_generator = data_generator(train_data, 24)
    test_generator = data_generator(test_data, 24)

    evaluator = Evaluator()
    history=model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=5, callbacks=[evaluator]
    )
    
    from tqdm import tqdm
    preall=[]
    trueall=[]
    for x_true, y_true in tqdm(test_generator):
            y_pred = model.predict(x_true).argmax(axis=1)
            preall.append(y_pred)
            trueall.append(np.squeeze(y_true,axis=1))
    preall=np.concatenate(preall,axis=0)
    trueall=np.concatenate(trueall,axis=0)
    print (classification_report(trueall,preall,digits=4,target_names=["IS","ES"]))
    true_labels_all.extend(np.argmax(y_test,axis=1))
    predicted_labels_all.extend(testpre)

D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9468    0.7461    0.8345      1646
          ES     0.5591    0.8848    0.6852       599

    accuracy                         0.7831      2245
   macro avg     0.7529    0.8154    0.7599      2245
weighted avg     0.8433    0.7831    0.7947      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9335    0.7392    0.8250      1614
          ES     0.5646    0.8653    0.6834       631

    accuracy                         0.7746      2245
   macro avg     0.7491    0.8022    0.7542      2245
weighted avg     0.8298    0.7746    0.7852      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9425    0.7407    0.8295      1639
          ES     0.5559    0.8779    0.6807       606

    accuracy                         0.7777      2245
   macro avg     0.7492    0.8093    0.7551      2245
weighted avg     0.8382    0.7777    0.7894      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9456    0.7253    0.8210      1631
          ES     0.5493    0.8893    0.6791       614

    accuracy                         0.7702      2245
   macro avg     0.7475    0.8073    0.7500      2245
weighted avg     0.8372    0.7702    0.7822      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9367    0.7214    0.8151      1601
          ES     0.5589    0.8787    0.6832       643

    accuracy                         0.7665      2244
   macro avg     0.7478    0.8001    0.7491      2244
weighted avg     0.8285    0.7665    0.7773      2244


              precision    recall  f1-score   support

          IS     0.9468    0.7461    0.8345      1646
          ES     0.5591    0.8848    0.6852       599

    accuracy                         0.7831      2245
   macro avg     0.7529    0.8154    0.7599      2245
weighted avg     0.8433    0.7831    0.7947      2245

              precision    recall  f1-score   support

          IS     0.9335    0.7392    0.8250      1614
          ES     0.5646    0.8653    0.6834       631

    accuracy                         0.7746      2245
   macro avg     0.7491    0.8022    0.7542      2245
weighted avg     0.8298    0.7746    0.78

In [ ]:
# 将结果转换为numpy数组
true_labels_all = np.array(true_labels_all)
predicted_labels_all = np.array(predicted_labels_all)

# 创建一个DataFrame
results_df = pd.DataFrame({
    'True_Label': true_labels_all,
    'Predicted_Label': predicted_labels_all
})

# 将DataFrame保存为CSV文件
results_df.to_csv('bert_cross_validation.csv', index=False)